In [ ]:
import keras
import numpy as np

from keras.datasets import reuters
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.xception import decode_predictions
from keras.applications.xception import preprocess_input
from matplotlib import pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
from scipy import misc
import glob

### Overconfidence of the network

In [ ]:
model = keras.applications.xception.Xception(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

CVPR 2015 - Deep Neural Networks are Easily Fooled: High Confidence Predictions for Unrecognizable Images

In [ ]:
# The deep neural network is the pre-trained network modeled on AlexNet provided by Caffe.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15, 45))

files = sorted(glob.glob("./example_images/out-of-distribution/*"))

for j, image_path in enumerate(files):
    image_ = image.load_img(image_path)
    
    image_preprocessed = preprocess_input(image.img_to_array(image_)[np.newaxis])
    
    preds = model.predict(image_preprocessed)
    
    y = j % 2
    x = j // 2
    axes[x, y].imshow(image_)
    img_, class_, prob_ = decode_predictions(preds)[0][0]
    axes[x, y].set_title(f'{class_} - probability {prob_}')

    j += 1


## Simple methods for discovering uncertainties

Now we will focus on a basic technique that can be used to:
+ Find out-if-distribution examples
+ Model aleatoric uncertainty

### K-nearest neighbours



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_blobs

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Gaussian divided into three quantiles", fontsize='large')
X1, Y1 = make_blobs(n_features=2, centers=3, cluster_std=2, random_state=9)
_ = plt.scatter(X1[:, 0], X1[:, 1], marker='o', c=Y1,
                s=50, edgecolor='k')

In [ ]:
classifier = KNeighborsClassifier()

In [ ]:
_ = classifier.fit(X1, Y1)

#### 1). Find out-of-distribution samples

For a point we compute the mean distance to k nearest neighbours.

Then, the distance serves as an out-of-distribution detection metric
 

In [ ]:
x_forgrid = np.linspace(-15, 2.5, 100)
y_forgrid = np.linspace(-10, 4, 100)

X_mesh, Y_mesh = np.meshgrid(x_forgrid, y_forgrid)

In [ ]:
coordinates = np.stack((X_mesh.flatten(), Y_mesh.flatten()), axis=1)

In [ ]:
grid_to_plot = classifier.kneighbors(coordinates)[0].mean(axis=1)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Out-of-distribution uncertainty", fontsize='large')
X1, Y1 = make_blobs(n_features=2, centers=3, cluster_std=2, random_state=9)

plt.scatter(
    coordinates[:, 0],
    coordinates[:, 1],
    s=80,
    alpha=0.9,
    c=-grid_to_plot,
    cmap='gray',
    marker='s'

)


_ = plt.scatter(X1[:, 0], X1[:, 1], marker='o', c=Y1,
                s=50, edgecolor='k')



We can compare an example from a test set with the training set under this metric, In case of a point from a training set, we will ignore the point itself.

In [ ]:
training_set_distances = classifier.kneighbors(X1)[0][:, 1:].mean(axis=1)

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(training_set_distances, bins=20, normed=True, alpha=0.5, label='Training set')
_ = plt.hist(grid_to_plot, bins=100, normed=True, alpha=0.5, label='All the gridpoints')
plt.legend()
_ = plt.xlabel('Average distance')

#### Find approximation of alleatoric uncertainty

Knowing that probabilities can give us basic measure of uncertainty, we will now create a simple measure of alleatoric uncertainty:

$\frac{\sum_{i}{d_{i}}}{k} - \lambda f(x)$

where 
+ $k$ - number of nearest neighbours
+ $d_{i}$ - distances of k nearest neighbours
+ $f$ - model - outputs the probability of the most likely class
+ $x$ - sample
+ $\lambda$ - hyperparameter


In [ ]:
grid_predictions = classifier.predict_proba(coordinates).max(axis=1)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Three blobs - all uncertainties", fontsize='large')
X1, Y1 = make_blobs(n_features=2, centers=3, cluster_std=2, random_state=9)

plt.scatter(
    coordinates[:, 0],
    coordinates[:, 1],
    s=80,
    alpha=0.9,
    c=np.clip(-grid_to_plot + 5 * grid_predictions, 0, None),
    cmap='gray',
    marker='s'

)

_ = plt.scatter(X1[:, 0], X1[:, 1], marker='o', c=Y1,
                s=50, edgecolor='k')


### Exercise

We will work on Reuters dataset.

I extract two out of all classes. The task is to create a method that maximizes ROC AUC in the out-of-distribution detection task. Out-of-distribution detection task is a binary classification problem where the classes are: out-of-distribution examples and in-distribution examples.

In [ ]:
'''Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
'''
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)

We will  use only 2 classes in the training set

In [ ]:
print(f"Before dataset cleaning: {y_train.max() + 1} classes")
x_train = x_train[y_train < 2]
y_train = y_train[y_train < 2]
print(f"After dataset cleaning: {y_train.max() + 1} classes")

In [ ]:

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
dt = LogisticRegression()

In [ ]:
dt.fit(x_train, y_train)

In [ ]:
### Definition of the target

In [ ]:
from sklearn.metrics import roc_auc_score
y_groundtruth = y_test > 1 # 1 is OOD example

In [ ]:
### Here you should write your code

In [ ]:
scores = None

In [ ]:
print(roc_auc_score(y_groundtruth, scores))